Have our own simple class to represent a note in time

In [137]:
import enum

class Note(enum.Enum):
    c = 0
    db = 1
    d = 2
    eb = 3
    e = 4
    f = 5
    gb = 6
    g = 7
    ab = 8
    a = 9
    bb = 10
    b = 11
    
    def __str__(self):
        if self == Note.c:
            return 'C'
        elif self == Note.db:
            return 'C#'
        elif self == Note.d:
            return 'D'
        elif self == Note.eb:
            return 'D#'
        elif self == Note.e:
            return 'E'
        elif self == Note.f:
            return 'F'
        elif self == Note.gb:
            return 'F#'
        elif self == Note.g:
            return 'G'
        elif self == Note.ab:
            return 'G#'
        elif self == Note.a:
            return 'A'
        elif self == Note.bb:
            return 'A#'
        elif self == Note.b:
            return 'B'

class NoteEvent:
    # Constructs event from midi.MidiEvent
    def __init__(self, midi_event, current_time):
        self.note = midi_event.note
        # In ticks
        self.start = midi_event.time + current_time
        self.end = None
    
    def is_valid(self):
        if self.note == None or self.octave == None:
            return False
        if self.start == None or self.end == None:
            return False
        if self.end - self.start <= 0:
            return False
        return True
    
    def get_note(self):
        return Note(self.note % 12)
    
    def get_octave(self):
        return self.note // 12

    def __str__(self):
        return str(self.get_note()) + str(self.get_octave()) + ' ' + str(self.start) + '-' + str(self.end)

class NoteEventSequence:
    # Helper function
    def __finish_note(self, msg, current_time):
        for event in self.note_events:
            if event.end != None:
                continue
            if event.note != msg.note:
                continue
            event.end = msg.time + current_time
            break
    
    # Constructs sequence from mido.MidiTrack
    def __init__(self, midi_track):
        self.note_events = []
        self.beats_per_measure = None
        current_time = 0
        for msg in midi_track:
            if msg.type == 'note_on':
                self.note_events.append(NoteEvent(msg, current_time))
            elif msg.type == 'note_off':
                self.__finish_note(msg, current_time)
            elif msg.type == 'time_signature':
                self.beats_per_measure = msg.numerator
            current_time += msg.time
    
    def __str__(self):
        s = 'NoteEventSequence:\n\n'
        for note_event in self.note_events:
            s += '\t(' + str(note_event) + ')'
        return s + '\n'
    
    def is_empty(self):
        return len(self.note_events) == 0

    def quantize(self, fraction, ticks_per_fraction):
        for note_event in self.note_events:
            note_event.start = int(round(note_event.start / ticks_per_fraction))
        
class Composition:
    def __init__(self, midi_file):
        self.tracks = []
        self.ticks_per_beat = midi_file.ticks_per_beat
        self.beats_per_measure = None
        for track in midi_file.tracks:
            note_event_sequence = NoteEventSequence(track)
            if note_event_sequence.beats_per_measure:
                self.beats_per_measure = note_event_sequence.beats_per_measure
            if not note_event_sequence.is_empty():
                self.tracks.append(note_event_sequence)
    
    def __str__(self):
        s = 'Composition: \n'
        s += '\tticks per beat: ' + str(self.ticks_per_beat) + '\n'
        s += '\tbeats per measure: ' + str(self.beats_per_measure) + '\n'
        for track in self.tracks:
            s += str(track) + '\n'
        return s

    # Quantize starts and ends to the nearest fraction beat (e.g., 8 to the nearest eights/quavers)
    def quantize(self, fraction):
        if self.beats_per_measure == None or self.ticks_per_beat == None:
            raise Exception('Invalid composition. Cannot quantize.')
        # Ticks per beat means ticks per quarter note (I think)
        ticks_per_fraction = 4 / fraction * self.ticks_per_beat
        for track in self.tracks:
            track.quantize(fraction, ticks_per_fraction)

We need to process each midi file. Which means loading the file, then for each track:

* Quantizing (quantize first in case the quantization process create unsuitable track)
* Discard from consideration if polyphonic
* Discard or sort separately if a simply classification algorithm (such as gaussiannb) determines it doesn't contain melodic information (drum part or too atonal to make sense of it)

In [138]:
import mido
import os
import re

def process_midi_file(filen):
    midi_file = mido.MidiFile(filen)
    composition = Composition(midi_file)
    composition.quantize(8)
    print(composition)

midi_file_re = re.compile('.midi?$', re.IGNORECASE)

from os.path import join, getsize
try:
    for root, dirs, files in os.walk('data'):
        for file in files:
            if (midi_file_re.search(file)):
                process_midi_file(os.path.join(root, file))
                raise BaseException
except Exception as e:
    print(e)
except BaseException:
    pass

Composition: 
	ticks per beat: 384
	beats per measure: 4
NoteEventSequence:

	(A4 0-256)	(F5 0-256)	(C6 6-1436)	(G5 6-1408)	(A#5 8-1504)	(A5 8-1792)	(F5 8-1792)	(A5 14-2972)	(C#5 14-2944)	(G5 16-3040)	(D5 16-3328)	(F5 16-3328)	(D5 20-3808)	(F5 20-3808)	(A#5 20-3808)	(D6 20-3808)	(A#5 20-4096)	(D5 20-4096)	(F5 20-4096)	(D6 20-4096)	(A#5 24-4604)	(E5 24-4604)	(F5 24-4864)	(A5 24-4864)	(A5 28-5372)	(G5 28-5420)	(A5 28-5468)	(G5 28-5516)	(A5 29-5564)	(G5 29-5612)	(A5 29-5660)	(G5 30-5708)	(A5 30-5756)	(G5 30-6044)	(F5 32-6112)	(F5 32-6524)	(A5 34-6908)	(C6 36-7292)	(F6 38-7552)	(F6 40-7868)	(D6 41-8000)	(D6 42-8636)	(A#5 45-8828)	(F5 46-9020)	(D5 47-9152)	(C6 48-9596)	(E5 50-9980)	(G5 52-10364)	(C6 54-10624)	(G#5 56-11132)	(A5 58-11392)	(B5 60-11900)	(D6 62-12160)	(C7 64-12544)	(G6 64-12544)	(E6 64-12544)	(C6 64-12544)	(C7 66-13184)	(G6 66-13184)	(E6 66-13184)	(A#6 70-13696)	(G#6 72-14204)	(A6 74-14464)	(A6 77-14972)	(B6 78-15164)	(C7 79-15356)	(C7 80-15548)	(D6 81-15740)	(D6 82-16128)	(E6